## import libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

## Data Load

In [ ]:
df = pd.read_excel('Sales Dataset.xlsx')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [ ]:
df.shape

(9994, 21)

In [ ]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9983 non-null   float64       
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

## Data Cleaning

<h3> Check Duplicates

In [ ]:
df.duplicated().sum()

np.int64(0)

<h3> Handle Missing Data

In [ ]:
df.isnull().sum()

,0
Row ID,0
Order ID,0
Order Date,0
Ship Date,0
Ship Mode,0
Customer ID,0
Customer Name,0
Segment,0
Country,0
City,0


Fill Null Values in Postal Code

In [ ]:
df['Postal Code'] = df.groupby('City')['Postal Code'].transform(lambda x: x.fillna(method='ffill'))

In [ ]:
df.isna().sum().sum()

np.int64(0)

<h3> Feature Engineering

In [ ]:
df['Order Month'] = df['Order Date'].dt.to_period('M')

In [ ]:
df['Order Month'][0]

Period('2017-11', 'M')

In [ ]:
df['Order_Year'] = df['Order Date'].dt.year

df['Order_Month'] = df['Order Date'].dt.month

df['Order_Day'] = df['Order Date'].dt.day

df['Order_DayOfWeek'] = df['Order Date'].dt.dayofweek

df['Shipping Duration'] = (df['Ship Date'] - df['Order Date']).dt.days


In [ ]:
df['Profit_Margin'] = df['Profit'] / df['Sales']

df['Discounted_Price'] = df['Sales'] * (1 - df['Discount'])

df['Sales_per_Quantity'] = df['Sales'] / df['Quantity']

df['Cumulative_Sales_Customer'] = df.groupby('Customer ID')['Sales'].cumsum()

df['Cumulative_Profit_Region'] = df.groupby('Region')['Profit'].cumsum()

## Data Exploration

In [ ]:
df.describe()

,Row ID,Order Date,Ship Date,Postal Code,Sales,Quantity,Discount,Profit,Order_Year,Order_Month,Order_Day,Order_DayOfWeek,Shipping Duration,Profit_Margin,Discounted_Price,Sales_per_Quantity,Cumulative_Sales_Customer,Cumulative_Profit_Region
count,9994.000000,9994,9994,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,2017-04-30 05:17:08.056834048,2017-05-04 04:17:20.304182528,55227.083350,229.858001,3.789574,0.156203,28.656896,2016.722233,7.809686,15.468381,2.987793,3.958475,0.120314,197.580421,60.919569,1886.273729,36306.226063
min,1.000000,2015-01-03 00:00:00,2015-01-07 00:00:00,1040.000000,0.444000,1.000000,0.000000,-6599.978000,2015.000000,1.000000,1.000000,0.000000,0.000000,-2.750000,0.088800,0.336000,1.080000,-2232.611600
25%,2499.250000,2016-05-23 00:00:00,2016-05-27 00:00:00,23223.000000,17.280000,2.000000,0.000000,1.728750,2016.000000,5.000000,8.000000,1.000000,3.000000,0.075000,14.336000,5.470000,448.013000,12549.836075
50%,4997.500000,2017-06-26 00:00:00,2017-06-29 00:00:00,56430.500000,54.490000,3.000000,0.200000,8.666500,2017.000000,9.000000,15.000000,3.000000,4.000000,0.270000,45.923200,16.270000,1243.019000,30651.743300
75%,7495.750000,2018-05-14 00:00:00,2018-05-18 00:00:00,90008.000000,209.940000,5.000000,0.200000,29.364000,2018.000000,11.000000,23.000000,5.000000,5.000000,0.362500,180.176475,63.940000,2553.187000,54244.420200
max,9994.000000,2018-12-30 00:00:00,2019-01-05 00:00:00,99301.000000,22638.480000,14.000000,0.800000,8399.976000,2018.000000,12.000000,31.000000,6.000000,7.000000,0.500000,17499.950000,3773.080000,25043.050000,108418.448900
std,2885.163629,NaN,NaN,32028.491225,623.245101,2.225110,0.206452,234.260108,1.123555,3.284654,8.748178,2.180270,1.747603,0.466754,539.045278,142.927440,2218.970462,28454.645288


In [ ]:
def ShowDetails() :
    global data
    for col in df.columns :
        print(f'for feature {col}')
        print(f'Number of Nulls is {df[col].isna().sum()}')
        print(f'Number of Unique Values {len(df[col].unique())}')
        print(f'First values of Unique {df[col].unique()[:]}')
        print('----------------------------------')

In [ ]:
ShowDetails()

for feature Row ID
Number of Nulls is 0
Number of Unique Values 9994
First values of Unique [   1    2    3 ... 9992 9993 9994]
----------------------------------
for feature Order ID
Number of Nulls is 0
Number of Unique Values 5009
First values of Unique ['CA-2017-152156' 'CA-2017-138688' 'US-2016-108966' ... 'CA-2015-110422'
 'CA-2018-121258' 'CA-2018-119914']
----------------------------------
for feature Order Date
Number of Nulls is 0
Number of Unique Values 1236
First values of Unique <DatetimeArray>
['2017-11-08 00:00:00', '2017-06-12 00:00:00', '2016-10-11 00:00:00',
 '2015-06-09 00:00:00', '2018-04-15 00:00:00', '2017-12-05 00:00:00',
 '2016-11-22 00:00:00', '2015-11-11 00:00:00', '2015-05-13 00:00:00',
 '2015-08-27 00:00:00',
 ...
 '2017-10-11 00:00:00', '2015-06-18 00:00:00', '2018-02-28 00:00:00',
 '2016-05-09 00:00:00', '2015-11-06 00:00:00', '2016-12-29 00:00:00',
 '2015-04-03 00:00:00', '2017-06-03 00:00:00', '2016-04-12 00:00:00',
 '2015-01-21 00:00:00']
Length: 1236, 

In [ ]:
df.to_csv('Edited Sales Dataset.csv', index=False)

## Visualization

**Top Performing Categories**

In [ ]:
# Group by Category and aggregate Sales and Profit
category_stats = df.groupby('Category')[['Sales', 'Profit']].sum().sort_values(by='Sales', ascending=False)

# Plot: Sales (bar chart) + Profit (line chart)
fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar plot for Total Sales
sns.barplot(x=category_stats.index, y=category_stats['Sales'], color='skyblue', ax=ax1)
ax1.set_ylabel('Total Sales', color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')
ax1.set_title('Top Performing Categories: Sales & Profit')

# Line plot for Total Profit on secondary y-axis
ax2 = ax1.twinx()
sns.lineplot(x=category_stats.index, y=category_stats['Profit'], color='darkgreen', marker='o', linewidth=2.5, ax=ax2)
ax2.set_ylabel('Total Profit', color='darkgreen')
ax2.tick_params(axis='y', labelcolor='darkgreen')

# Beautify layout
plt.tight_layout()
plt.show()


**Discount Analysis by Category**

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='Category', y='Discount')
plt.title("Discount Levels Across Categories")

**Top 10 Products by Sales**

In [ ]:
top_products = df.groupby('Product Name')['Sales'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 6))
top_products.plot(kind='barh', color='skyblue')
plt.title('Top 10 Products by Sales')
plt.xlabel('Total Sales')
plt.ylabel('Product Name')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

**Region-wise Profitability**

In [ ]:
region_profit = df.groupby('Region')['Profit'].sum().sort_values()
region_profit.plot(kind='bar', title='Total Profit by Region', figsize=(8, 5))

**Best Cities by Sales**

In [ ]:
top_cities = df.groupby('City')['Sales'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_cities.values, y=top_cities.index, palette='Blues_d')
plt.title('Top 10 Cities by Total Sales')
plt.xlabel('Total Sales')
plt.ylabel('City')
plt.tight_layout()
plt.show()

**Profit vs. Discount Analysis**

In [ ]:
sns.scatterplot(data=df, x='Discount', y='Profit')

**Shipping Performance**

In [ ]:
df['Shipping Duration'].hist()

**Customer Segment Performance**

In [ ]:
segment_sales = df.groupby('Segment')['Sales'].sum()
plt.figure(figsize=(6, 6))
segment_sales.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
plt.title('Sales Share by Customer Segment')
plt.ylabel('')
plt.tight_layout()
plt.show()

**Customer Loyalty - Top Customers by Number of Orders**

In [ ]:
top_customers = df.groupby('Customer Name')['Order ID'].nunique().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 6))
top_customers.plot(kind='bar', color='purple')
plt.title('👥 Top 10 Most Loyal Customers (By Number of Orders)')
plt.xlabel('Customer Name')
plt.ylabel('Number of Unique Orders')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Top Customers by Sales**

In [ ]:
top_customers_sales = df.groupby('Customer Name')['Sales'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=top_customers_sales.values, y=top_customers_sales.index, palette='viridis')
plt.title('Top 10 Customers by Total Sales')
plt.xlabel('Total Sales')
plt.ylabel('Customer Name')
plt.tight_layout()
plt.show()

**Time Series of Profit vs. Sales**

In [ ]:
monthly = df.groupby(df['Order Date'].dt.to_period('M'))[['Sales', 'Profit']].sum()
monthly.plot(marker='o', figsize=(12, 6))
plt.title('Monthly Sales vs. Profit')
plt.xlabel('Month')
plt.ylabel('Amount')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
yearly = df.groupby(df['Order_Year'])[['Sales', 'Profit']].sum()
yearly.plot(marker='o', figsize=(12, 6))
plt.title('Yearly Sales vs. Profit')
plt.xlabel('Year')
plt.ylabel('Amount')
plt.grid(True)
plt.xticks(yearly.index)
plt.tight_layout()
plt.show()

**Heatmap of Profit by State**

In [ ]:
state_profit = df.pivot_table(values='Profit', index='State', aggfunc='sum').sort_values(by='Profit', ascending=False)
plt.figure(figsize=(8, 18))
sns.heatmap(state_profit, annot=True, fmt=".0f", cmap="YlGnBu", linewidths=0.5)
plt.title('Profit by State (Heatmap)')
plt.tight_layout()
plt.show()

**Loss-Making Orders (Top 10 most loss-making orders)**

In [ ]:
loss_orders = df[df['Profit'] < 0].sort_values(by='Profit').head(10)
plt.figure(figsize=(10, 6))
sns.barplot(data=loss_orders, x='Order ID', y='Profit', palette='Reds_r')
plt.title('Top 10 Loss-Making Orders')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**High Discount, Low Profit Cluster**

In [ ]:
high_discount_loss = df[(df['Discount'] > 0.5) & (df['Profit'] < 0)]
plt.figure(figsize=(10, 6))
sns.scatterplot(data=high_discount_loss, x='Discount', y='Profit', hue='Category')
plt.title('⚠️ High Discount & Low Profit Items')
plt.xlabel('Discount')
plt.ylabel('Profit')
plt.grid(True)
plt.tight_layout()
plt.show()

**Drop unnecessary columns**

In [ ]:
df.drop(columns=['Row ID','Order ID','Customer ID','Customer Name','Product ID','Product Name','Country','Postal Code', 'Order Date','Ship Date', 'Order Month'], axis=1, inplace=True)

In [ ]:
df.info()

**Encode categorical features**

In [ ]:
label_cols = ['City', 'State', 'Sub-Category']
for col in label_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

df = pd.get_dummies(df, columns=['Ship Mode', 'Segment', 'Region', 'Category'], drop_first=True)

## Split Data

In [ ]:
X = df.drop('Sales', axis=1)
y = df['Sales']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Data Scaling

In [ ]:
numeric_cols = [
    'Quantity', 'Discount', 'Profit',
    'Profit_Margin', 'Discounted_Price',
    'Sales_per_Quantity', 'Cumulative_Sales_Customer',
    'Cumulative_Profit_Region',
]

scaler = MinMaxScaler()

X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])

X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

## ML Models

In [ ]:
# Store model names and R2 scores
model_names = []
train_scores = []
test_scores = []

models = [
    ("Linear Regression", LinearRegression()),
    ("XGBoost", XGBRegressor(verbosity=0)),
    ("Decision Tree", DecisionTreeRegressor()),
    ("Random Forest", RandomForestRegressor()),
    ("KNN", KNeighborsRegressor()),
    ("SVM", SVR())
]

# Fit and evaluate each model
for name, model in models:
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print(f"--- {name} ---")
    print("Training MAE:", (mean_absolute_error(y_train, y_train_pred)))
    print("Testing MAE:", (mean_absolute_error(y_test, y_test_pred)))
    print("Training MSE:", (mean_squared_error(y_train, y_train_pred)))
    print("Testing MSE:", (mean_squared_error(y_test, y_test_pred)))
    print("Training R2:", (r2_score(y_train, y_train_pred)*100))
    print("Testing R2:", (r2_score(y_test, y_test_pred)*100))
    print("="*40)

    model_names.append(name)
    train_scores.append(r2_score(y_train, y_train_pred))
    test_scores.append(r2_score(y_test, y_test_pred))


In [ ]:
# === Polynomial Features for Ridge & Lasso ===
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# === Ridge Regression ===
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_poly, y_train)
y_train_pred_ridge = ridge.predict(X_train_poly)
y_test_pred_ridge = ridge.predict(X_test_poly)

model_names.append("Ridge Regression")
train_scores.append(r2_score(y_train, y_train_pred_ridge))
test_scores.append(r2_score(y_test, y_test_pred_ridge))

print("--- Ridge Regression ---")
print("Training MAE:", mean_absolute_error(y_train, y_train_pred_ridge))
print("Testing MAE:", mean_absolute_error(y_test, y_test_pred_ridge))
print("Training MSE:", mean_squared_error(y_train, y_train_pred_ridge))
print("Testing MSE:", mean_squared_error(y_test, y_test_pred_ridge))
print("Training R2:", r2_score(y_train, y_train_pred_ridge))
print("Testing R2:", r2_score(y_test, y_test_pred_ridge))
print("="*40)

# === Lasso Regression ===
lasso = Lasso(alpha=0.001, max_iter=10000)
lasso.fit(X_train_poly, y_train)
y_train_pred_lasso = lasso.predict(X_train_poly)
y_test_pred_lasso = lasso.predict(X_test_poly)

model_names.append("Lasso Regression")
train_scores.append(r2_score(y_train, y_train_pred_lasso))
test_scores.append(r2_score(y_test, y_test_pred_lasso))

print("--- Lasso Regression ---")
print("Training MAE:", mean_absolute_error(y_train, y_train_pred_lasso))
print("Testing MAE:", mean_absolute_error(y_test, y_test_pred_lasso))
print("Training MSE:", mean_squared_error(y_train, y_train_pred_lasso))
print("Testing MSE:", mean_squared_error(y_test, y_test_pred_lasso))
print("Training R2:", r2_score(y_train, y_train_pred_lasso))
print("Testing R2:", r2_score(y_test, y_test_pred_lasso))
print("="*40)

In [ ]:
x = range(len(model_names))
plt.figure(figsize=(12, 6))
plt.bar(x, train_scores, width=0.4, label='Train R²', align='center', color='skyblue')
plt.bar([p + 0.4 for p in x], test_scores, width=0.4, label='Test R²', align='center', color='orange')
plt.xticks([p + 0.2 for p in x], model_names, rotation=45)
plt.title('Train vs Test R² Scores for Regression Models')
plt.ylabel('R² Score')
plt.legend()
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()